In [1]:
import pandas as pd
from PIL import Image
import numpy as np
import re
import fileinput
import pickle
import os
# import skimage.io
# import skimage.transform
# from keras.preprocessing.text import Tokenizer, one_hot

In [2]:
tags = pd.read_csv('~/Dataset/anime/tags_clean.csv', header=None, sep='\t', names=list(range(50)))
print(len(tags))
clean = tags.applymap(lambda x:re.sub('[\d,:""]','', str(x)))
mask = clean.applymap(lambda x:("eyes" in str(x)) or ("hair" in str(x)))
clean = clean.where(mask)

/home/dchenam/miniconda3/envs/pytorch/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (43,44,45,46,47,48,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


33431


In [3]:
clean.to_csv('tags_clean.txt', header=None, sep='\t')

In [4]:
x = fileinput.input('tags_clean.txt', inplace=1)
for line in x:
    line = re.sub('\t', ' ', line)
    line = re.sub("long hair", ' ', line)
    line = re.sub("short hair", ' ', line)
    if not ('hair' in line) and not ('eye' in line):
        line = ''
    if line.count('hair') > 1:
        line = ''
    if line.count('eyes') > 1:
        line = ''
    line = re.sub("pubic hair", ' ', line)
    line = re.sub("rutherford", ' ', line)
    line = re.sub("  eyes  ", ' ', line)
    line = re.sub("  hair  ", ' ', line)
    line = re.sub(' hair', '_hair', line)
    line = re.sub(' eye', '_eye', line)
    line = re.sub('\s{2,}', ' ', line.strip())
    if re.match(r'^\s*$', line):
        continue
    print(line.strip())
x.close()

In [212]:
data = []
with open('tags_clean.txt', 'r') as f:
    data.extend(line for line in f)

In [216]:
idx = []
captions = []
for line in data:
    idx.append(int(re.findall(r'\d+', line)[0]))
    captions.append(re.sub("\d+\s", "", line.strip()))

In [7]:
img_data = []
for i in idx:
    img = skimage.io.imread(os.path.join('data/faces', str(i) + '.jpg'))
    img = skimage.transform.resize(img, (64, 64))
    img_data.append(img)

/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [9]:
img_data = np.array(img_data)
img_data = img_data * 2 - 1

In [73]:
np.save('train_images.npy', img_data)

In [3]:
import os.path
from skip_thoughts import configuration
from skip_thoughts import encoder_manager

In [4]:
VOCAB_FILE = "skip_thoughts_uni/vocab.txt"
EMBEDDING_MATRIX_FILE = "skip_thoughts_uni/embeddings.npy"
CHECKPOINT_PATH = "skip_thoughts_uni/"

In [5]:
encoder = encoder_manager.EncoderManager()
encoder.load_model(configuration.model_config(),
                   vocabulary_file=VOCAB_FILE,
                   embedding_matrix_file=EMBEDDING_MATRIX_FILE,
                   checkpoint_path=CHECKPOINT_PATH)

INFO:tensorflow:Reading vocabulary from skip_thoughts_uni/vocab.txt
INFO:tensorflow:Loaded vocabulary with 930914 words.
INFO:tensorflow:Loading embedding matrix from skip_thoughts_uni/embeddings.npy
INFO:tensorflow:Loaded embedding matrix with shape (930914, 620)
INFO:tensorflow:Building model.
INFO:tensorflow:Loading model from checkpoint: skip_thoughts_uni/model.ckpt-501424
INFO:tensorflow:Restoring parameters from skip_thoughts_uni/model.ckpt-501424
INFO:tensorflow:Successfully loaded checkpoint: model.ckpt-501424


In [10]:
encodings = encoder.encode(caption)

In [11]:
encodings = np.array(encodings)
np.save('train_embeddings.npy', encodings)

In [12]:
import scipy.spatial.distance as sd
def get_nn(ind, num=10):
  encoding = encodings[ind]
  scores = sd.cdist([encoding], encodings, "cosine")[0]
  sorted_ids = np.argsort(scores)
  print("Sentence:")
  print("", caption[ind])
  print("\nNearest neighbors:")
  for i in range(1, num + 1):
    print(" %d. %s (%.3f)" %
          (i, caption[sorted_ids[i]], scores[sorted_ids[i]]))

In [197]:
d = {'orange hair':'1', 'white hair':'2', 'aqua hair':'3', 'gray hair':'4',
    'green hair':'5', 'red hair':'6', 'purple hair':'7', 'pink hair':'8',
    'blue hair':'9', 'black hair':'10', 'brown hair':'11', 'blonde hair':'12','gray eyes': '13',
    'black eyes':'14', 'orange eyes':'15', 'blue eyes': '24',
    'pink eyes':'16', 'yellow eyes':'17', 'aqua eyes':'18', 'purple eyes':'19',
    'green eyes':'20', 'brown eyes':'21', 'red eyes':'22', 'bicolored eyes':'23'}

In [168]:
testing_data = []
for hair in color_hairs:
    for eye in color_eyes:
        testing_data.append(hair + ' ' + eye)
        testing_data.append(eye + ' ' + hair)
for hair in color_hairs:
    testing_data.append(hair)
for eye in color_eyes:
    testing_data.append(eye)

In [198]:
pattern = re.compile('|'.join(d.keys()))

In [220]:
for i, caption in enumerate(captions):
    result = pattern.sub(lambda x: d[x.group()], caption)
    try:
        caption = [int(s) for s in result.split(' ')]
    except:
        print(i, result)

1599 eyes
3695 pubic hair 19
4555 eyes
6156 eyes 6
7016 eyes
7403 eyes rutherford
10002 eyes rutherford
11437 eyes
13655 eyes
14214 eyes


In [205]:
captions

[]

In [146]:
tokenized_captions = tokenizer.texts_to_matrix(caption)

In [152]:
np.save('train_captions.npy', tokenized_captions)

In [34]:
testing_encoding = encoder.encode(testing_data)

In [163]:
testing_encoding = tokenizer.texts_to_matrix(testing_data)

In [36]:
testing_dict = dict(zip(testing_data, testing_encoding))

In [37]:
testing_dict['aqua eyes blonde hair']

array([ -1.04750069e-02,  -6.59986362e-02,  -4.85626333e-05, ...,
         5.74678509e-03,  -6.31452072e-04,  -2.37361155e-02], dtype=float32)

In [40]:
import scipy.spatial.distance as sd
def get_nn(ind, num=10):
  encoding = testing_encoding[ind]
  scores = sd.cdist([encoding], encodings, "cosine")[0]
  sorted_ids = np.argsort(scores)
  print("Sentence:")
  print("", testing_data[ind])
  print("\nNearest neighbors:")
  for i in range(1, num + 1):
    print(" %d. %s (%.3f)" %
          (i, filtered_data[sorted_ids[i]], scores[sorted_ids[i]]))

In [47]:
get_nn(13)

Sentence:
 brown eyes orange hair

Nearest neighbors:
 1. brown eyes orange hair (0.000)
 2. brown eyes orange hair (0.000)
 3. brown eyes orange hair (0.000)
 4. brown eyes orange hair (0.000)
 5. brown eyes orange hair (0.000)
 6. brown eyes orange hair (0.000)
 7. green eyes orange hair (0.036)
 8. green eyes orange hair (0.036)
 9. green eyes orange hair (0.036)
 10. green eyes orange hair (0.036)


In [85]:
test_embeddings = pickle.dump(testing_dict, open('test_embeddings.pkl', 'wb'))

In [117]:
a = np.random.rand(12288)

In [118]:
a = a.reshape(64, 64, 3)

In [119]:
a *= 255

In [120]:
a = a.astype(int)

In [124]:
import skimage.io
skimage.io.imsave('test.jpg', a)

/usr/local/lib/python3.5/dist-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from int64 to float64
  .format(dtypeobj_in, dtypeobj_out))
/usr/local/lib/python3.5/dist-packages/skimage/io/_io.py:132: UserWarning: test.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.5/dist-packages/skimage/util/dtype.py:118: UserWarning: Possible sign loss when converting negative image of type int64 to positive image of type uint8.
  .format(dtypeobj_in, dtypeobj_out))
/usr/local/lib/python3.5/dist-packages/skimage/util/dtype.py:171: UserWarning: Downcasting int64 to uint8 without scaling because max value 254 fits in uint8
  "value {} fits in {}".format(a.dtype, dtype, a.max(), dtype))


In [3]:
test_embeddings = pickle.load(open('test_embeddings.pkl', 'rb'))

In [10]:
image_data = np.load('train_images.npy')

In [12]:
batch_size = 64

In [16]:
random_idx = np.random.randint(len(test_embeddings), size=batch_size)

In [23]:
random_idx.shape

(64,)

In [40]:
np.array(list(test_embeddings.values()))[random_idx]

array([[-0.0155213 , -0.06247005,  0.00972278, ..., -0.00265473,
        -0.00262971, -0.0101154 ],
       [ 0.00730582, -0.06180365,  0.02337626, ..., -0.00276405,
        -0.00388468, -0.00573695],
       [ 0.00535974, -0.06033414,  0.02247241, ...,  0.00122462,
        -0.00298307,  0.00044312],
       ..., 
       [ 0.00701852, -0.06228148,  0.022693  , ..., -0.00914285,
         0.00590803, -0.00861985],
       [-0.01162465, -0.05671365,  0.02520377, ...,  0.00193969,
         0.00093961, -0.01516995],
       [ 0.00578264, -0.05740115,  0.01656133, ..., -0.00552971,
        -0.00404952, -0.01117698]], dtype=float32)